# Pandas基础

该部分主要参考[Pandas中文网](https://www.pypandas.cn/docs/)和[官方用户手册](https://pandas.pydata.org/pandas-docs/stable/index.html)，以及 [Pandas教程](https://www.yiibai.com/pandas)

Pandas是一个开源的，为Python编程语言提供高性能，易于使用的**数据结构**和**数据分析工具**。在hydrus环境下安装pandas：

```Shell
conda install -c conda-forge pandas
```

Pandas适合处理多种不同类型的数据：

- 不同类型的数据列组成的表格数据；
- 有序或无序的时间序列数据；
- 带行列标签的类型相同或不同的行或列组成的矩阵数据；
- 任意形式的观测或统计数据，它们不需标记即可放入pandas数据结构中。 

其运算都主要都围绕两类数据结构展开，Series和Dataframe，接下来从它们开始记录。

Series(1维)和DataFrame(2维)能处理绝大多数的典型应用，包括金融、统计、社科、工程等各个领域。
 
对于R用户，DataFrame提供了所有R的data.frame提供的，并且还有更多。

pandas建立在NumPy之上，也整合了很多第三方库，打造了良好的科学计算环境。

pandas处理的经典问题包括且不限于：

- 处理缺失数据，用NaN表示；
- 数据结构大小可变：可以插入删除DataFrame或者更高维对象的列；
- 自动且明确的data alignment，可以明确指定数据到一系列标签下；
- 灵活数据分组，很容易聚合转换数据；
- 容易将杂乱、标签复杂的Python和NumPy数据结构转换为DataFrame对象；
- 根据标签容易slicing,indexing和subsetting大数据集；
- 容易合并和连接数据集；
- 灵活地reshaping和pivoting数据集；
- 多级标签；
- 从各类文件中加载数据, 包括HDF5格式的数据；
- 时间序列指定的功能：日期范围生成和频率转换,滑动窗口的统计, 滑动线性回归, 日期平移或滞后等。

在数据处理的各个环节，包括变换和清洗数据，分析建模，组织分析结果为正式的格式，可视化和输出数据等，pandas都能很好地处理。

接下来先记录pandas基本数据结构，然后日常积累常见数据处理操作。

## Pandas基本数据结构

最好的理解Pandas数据结构的方式是将其当做其更低维数据的容器，比如Series是标量数据的容器，Dataframe是Series的容器。
可以通过类似字典的方式从容器中插入或移除对象。

pandas中为了更直观地给出数据结构不同维度的信息，更多用index和columns，而不是使用axis0和1这样来表示。例如：

``` python
for col in df.columns:
    series = df[col]
    # do something with series
```

pandas所有数据结构都是值可变的，但不总是size可变的。比如Series的长度是不可变的，但是Dataframe中是可以插入列的。
绝大多数方法都会产生新的对象，而不改变原输入数据。一般来说，pandas喜欢保持不可变性。

基本上来说，是利用numpy的数组为值依据，用columns和index给列和行起名。

In [2]:
import numpy as np
import pandas as pd
# 初始化Series
k = pd.Series({'a':np.random.randint(10,size=5),'b':0})
k

a    [0, 0, 2, 8, 0]
b                  0
dtype: object

In [3]:
# 检索series的项
print(k.iloc[1:])
print(k["b"])

b    0
dtype: object
0


In [4]:
# 另一种常用的初始化Series的方式
s = pd.Series([1, 2, 3, 4], index=['A', 'B', 'C', 'D'])
s

A    1
B    2
C    3
D    4
dtype: int64

In [5]:
# 创建一个空的 DataFrame
df_empty = pd.DataFrame(columns=['A', 'B', 'C', 'D'])
print(df_empty)
print("列名：",df_empty.columns.values)
# 创建一个只有一行的，不能使用下面的方式
# df_1r = pd.DataFrame(np.array([[1],[2],[3],[4]]),columns=['A', 'B', 'C', 'D'])
# 需要使用append：先定一个空的，再来添加
df_1r = df_empty
df_1r.loc[0] = [1,2,3,4]
print("有一行：\n",df_1r)
# 创建有行有列的
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
print(df)
# 使用numpy数组，命名行列进行初始化
df2 = pd.DataFrame(np.arange(16).reshape((4,4)), columns=['one', 'two', 'three', 'four'], index=['a', 'b', 'c','d'])
print(df2)
# 带时间序列的dataframe的初始化
time_range = pd.date_range('2011-1-1', periods=4, freq='D')
df2.index = time_range
print(df2)
print('列：\n',df2.columns)
print('列名称：\n',df2.columns.values)
time_range = pd.date_range('2011-1-1', '2011-1-4', freq='D')
df2.columns = time_range
print(df2)

Empty DataFrame
Columns: [A, B, C, D]
Index: []
列名： ['A' 'B' 'C' 'D']
有一行：
    A  B  C  D
0  1  2  3  4
   a  b  c
0  0  0  0
1  1  1  1
2  2  2  2
   one  two  three  four
a    0    1      2     3
b    4    5      6     7
c    8    9     10    11
d   12   13     14    15
            one  two  three  four
2011-01-01    0    1      2     3
2011-01-02    4    5      6     7
2011-01-03    8    9     10    11
2011-01-04   12   13     14    15
列：
 Index(['one', 'two', 'three', 'four'], dtype='object')
列名称：
 ['one' 'two' 'three' 'four']
            2011-01-01  2011-01-02  2011-01-03  2011-01-04
2011-01-01           0           1           2           3
2011-01-02           4           5           6           7
2011-01-03           8           9          10          11
2011-01-04          12          13          14          15


可以初始化一个只有列名的空Dataframe，然后再给列赋新值，可以使用如下方式

In [6]:
import pandas as pd
df0 = pd.DataFrame(columns=['A', 'B', 'C', 'D'])
df0["A"] = np.arange(5)
df0

,A,B,C,D
0,0,NaN,NaN,NaN
1,1,NaN,NaN,NaN
2,2,NaN,NaN,NaN
3,3,NaN,NaN,NaN
4,4,NaN,NaN,NaN


如果想要批次给各列赋值，可以这样：

In [7]:
for column in df0.columns:
    df0[column]=np.arange(5)
df0

,A,B,C,D
0,0,0,0,0
1,1,1,1,1
2,2,2,2,2
3,3,3,3,3
4,4,4,4,4


有时候会需要指定某些列的数据类型，这时候可以使用如下方式：

In [8]:
a = [['a', '1.2', '4.2'], ['b', '70', '0.03'], ['x', '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])
df

,one,two,three
0,a,1.2,4.2
1,b,70,0.03
2,x,5,0


In [9]:
df.dtypes

one      object
two      object
three    object
dtype: object

In [10]:
df[['two', 'three']] = df[['two', 'three']].astype(float)
df.dtypes

one       object
two      float64
three    float64
dtype: object

pandas与numpy数据结构之间可以方便地进行转换。

pandas转为numpy比较简单，直接调用value即转为ndarray。反过来，也很容易，直接用pandas的类型，相当于直接初始化了。

In [11]:
# pandas dataframe与numpy array之间的转换
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
print(df)
# df转换为ndarray
array_from_df=df.values
print(array_from_df)
print(np.array(df))
# 读取某一列数据，两种方式均可
print(df['A'])
print(df.loc[:, 'A'])
# 一列数据转换为ndarray
print(np.array(df['A']))
# Series转ndarray
# Creating the Series
sr = pd.Series(['New York', 'Chicago', 'Toronto', 'Lisbon', 'Rio'])
# Create the Index
index_ = ['City 1', 'City 2', 'City 3', 'City 4', 'City 5']
# set the index
sr.index = index_
# return numpy array representation
result = sr.values
# Print the result
print("series转为ndarray：")
print(result)
# Print the series
print(sr)

   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9
[[1 4 7]
 [2 5 8]
 [3 6 9]]
[[1 4 7]
 [2 5 8]
 [3 6 9]]
0    1
1    2
2    3
Name: A, dtype: int64
0    1
1    2
2    3
Name: A, dtype: int64
[1 2 3]
series转为ndarray：
['New York' 'Chicago' 'Toronto' 'Lisbon' 'Rio']
City 1    New York
City 2     Chicago
City 3     Toronto
City 4      Lisbon
City 5         Rio
dtype: object


## IO工具

很多时候，初始化DataFrame都是从读取文件开始的。对于csv文件和txt文件，都可以通过pandas的read_csv()函数读取。

In [12]:
from io import StringIO, BytesIO
import pandas as pd
data = ('col1,col2,col3\n'
         'a,b,01\n'
         'a,b,02\n'
         'c,d,03')
print("原数据：\n",data)
print("pandas读取默认设置：\n",pd.read_csv(StringIO(data)))
# 读取数据的时候即进行slice
print("pandas读取前两列：\n",pd.read_csv(StringIO(data), usecols=range(0,2)))
print("pandas读取某两列：\n",pd.read_csv(StringIO(data), usecols=lambda x: x.upper() in ['COL1', 'COL3']))
print("pandas读取某些行：\n",pd.read_csv(StringIO(data), skiprows=lambda x: x % 2 != 0))
print("读取数据之后，重新命名列名：\n")
df_example=pd.read_csv(StringIO(data))
df_example.columns = ['A','B','C']
df_example

原数据：
 col1,col2,col3
a,b,01
a,b,02
c,d,03
pandas读取默认设置：
   col1 col2  col3
0    a    b     1
1    a    b     2
2    c    d     3
pandas读取前两列：
   col1 col2
0    a    b
1    a    b
2    c    d
pandas读取某两列：
   col1  col3
0    a     1
1    a     2
2    c     3
pandas读取某些行：
   col1 col2  col3
0    a    b     2
读取数据之后，重新命名列名：



,A,B,C
0,a,b,1
1,a,b,2
2,c,d,3


另外，有时候会需要指定某一列作为dataframe的index，比如指定第一列为index，可以使用如下形式：

In [13]:
data = ('col1,col2,col3\n'
         'a,b,01\n'
         'a,b,02\n'
         'c,d,03')
print("原数据：\n",data)
print("pandas读取默认设置：\n",pd.read_csv(StringIO(data), index_col=0))

原数据：
 col1,col2,col3
a,b,01
a,b,02
c,d,03
pandas读取默认设置：
      col2  col3
col1           
a       b     1
a       b     2
c       d     3


专业方面，很多数据都是把年月或者年旬或者年日分别当做行和列，在计算时，需要进行处理，把时间变为一列或一行，即把几列或行的数组拼接起来
即将[1 2;2 3]的数据变为[1 2 2 3]，pandas的concat和numpy的concatenate类似。最后把行名index统一换成日期，构成时间序列Series

In [14]:
# 如果遇到“UnicodeDecodeError: 'utf8' codec can't decode byte....”错误，用记事本另存csv文件时，将“编码”设置为‘UTF-8’即可
dataset = pd.read_csv('Sheet1.csv')
# header指定某行的值作为各列的列名，1表示第二行，如果是None，则从第一行开始就是数据。
dataset = pd.read_csv('Sheet1.csv', header=1)
# 因为给定的数据格式，各人有各自的一套，所以具体情况具体分析。比如，这里dataset的第一列可以去掉，即得到所有数据，删除列时，用drop函数，加参数axis=1，不加则表示删除行
dataset1 = dataset.drop(['旬平均'], axis=1)
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
# iloc获取的数据格式为Series
se = df.iloc[:, 0]
# 获取dataframe的列数：df.shape[1]
for i in range(1, df.shape[1]):
    se_temp = df.iloc[:, i]
    se = pd.concat([se, se_temp])
# 如果一开始没有给series起名，比如从csv读取出来时是dataframe，拼接之后没办法再给Series起名，但是又需要给它起名，那么只能采取重新构造一个series的方式来命名
se = pd.Series(se, name="aaaaaaa")
# 把每行名称换为日期
rng = pd.date_range('2011-1-1', periods=9, freq='H')
se.index = rng
se = pd.Series(se, name="bbbbb")
se.head()

2011-01-01 00:00:00    0
2011-01-01 01:00:00    1
2011-01-01 02:00:00    2
2011-01-01 03:00:00    0
2011-01-01 04:00:00    1
Freq: H, Name: bbbbb, dtype: int64

当csv中包含了大量的编号代码，比如是002开头的编号——002111， 使用pd.read_csv('text.csv') 则会让所有的002xxx，变成了2xxx，前面2个0不见了，因为作为数值类型，没有必要保留0。

因此，为了保持编号的字符串特性，直接在参数一栏设置一下即可：
df＝pd.read_csv('text.csv', dtype={'code':str}
 
这样，把你要转换的列的名字设定好， “code”列中的数据读取为str，用列名或者列序号均可。
 
这样，读取到的数据就是按照我们的要求的了。

In [15]:
dataset = pd.read_csv(StringIO(data),dtype={2:str})
print(dataset)
dataset = pd.read_csv(StringIO(data),dtype={'col3':str})
print(dataset)

  col1 col2 col3
0    a    b   01
1    a    b   02
2    c    d   03
  col1 col2 col3
0    a    b   01
1    a    b   02
2    c    d   03


如果文件有注释行，读取的时候可以使用comment参数跳过

In [16]:
import pandas as pd
file= 'test-comment.csv'
data_wo_comment = pd.read_csv(file, comment='#')
data_wo_comment

,col1,col2,col3
0,A,2,0
1,A,1,1
2,B,9,9
3,NaN,8,4
4,D,7,2
5,C,4,3


读取excel文件的话稍微有些不同，这时候我们需要使用read_excel函数，该函数需要额外的 openpyxl 包，因此安装此包：

```Shell
conda install -c conda-forge openpyxl
```

In [19]:
import pandas as pd
df = pd.read_excel('AK_U.xlsx',sheet_name='AK')
# df = pd.read_excel('NID2018_U.xlsx')
df.head()

,RECORDID,DAM_NAME,OTHER_DAM_NAME,DAM_FORMER_NAME,STATEID,NIDID,LONGITUDE,LATITUDE,SECTION,COUNTY,...,CONG_NAME,PARTY,CONG_DIST,OTHERSTRUCTUREID,NUMSEPARATESTRUCTURES,PERMITTINGAUTHORITY,INSPECTIONAUTHORITY,ENFORCEMENTAUTHORITY,JURISDICTIONALDAM,EAP_LAST_REV_DATE
0,1,COOPER LAKE,NaN,NaN,NaN,AK00001,-149.823147,60.433708,NaN,KENAI PENINSULA,...,Don Young (R),R,AK00,NaN,1,N,N,N,N,08-JAN-18
1,2,BLUE LAKE,NaN,NaN,NaN,AK00002,-135.191700,57.063300,"T55S, R64E, S35",SITKA,...,Don Young (R),R,AK00,NaN,1,N,N,N,N,27-DEC-17
2,3,SALMON CREEK,NaN,NaN,NaN,AK00003,-134.403608,58.341850,"T41S,R67E,S2",JUNEAU,...,Don Young (R),R,AK00,NaN,1,N,N,N,N,27-SEP-17
3,4,ANNEX CREEK,NaN,NaN,NaN,AK00004,-134.126578,58.326939,"T41S,R69E,S9",JUNEAU,...,Don Young (R),R,AK00,NaN,2,N,N,N,N,27-SEP-17
4,5,CRYSTAL LAKE,NaN,NaN,NaN,AK00005,-132.845500,56.600000,"T61S,R81E,S6",WRANGELL-PETERSBURG,...,Don Young (R),R,AK00,NaN,1,N,N,N,N,19-DEC-17


另外，有时候还会使用 feather 文件，feather 是一种高效读写二进制数据的文件格式。不过根据https://zhuanlan.zhihu.com/p/69221436 一文中的建议，pandas自带的to_feather和read_feather容易有版本不兼容的问题，所以最好还是使用feather自带的api。安装feather：

```Shell
conda install -c conda-forge feather-format
```

首先看个例子，把csv文件转为feather文件

In [21]:
import numpy as np 
import pandas as pd
import os
import feather

In [22]:
PATH = 'attr_temp99%_days_99sites.csv'
df_temp = pd.read_csv(PATH)
df_temp.head()

,Unnamed: 0,DRAIN_SQKM,PPTAVG_BASIN,T_AVG_BASIN,T_MAX_BASIN,T_MAXSTD_BASIN,T_MIN_BASIN,T_MINSTD_BASIN,RH_BASIN,STREAMS_KM_SQ_KM,...,CONTACT,CANALS_PCT,RAW_AVG_DIS_ALLCANALS,NPDES_MAJ_DENS,RAW_AVG_DIS_ALL_MAJ_NPDES,FRESHW_WITHDRAWAL,PCT_IRRIG_AG,ROADS_KM_SQ_KM,PADCAT1_PCT_BASIN,PADCAT2_PCT_BASIN
0,0,78.68936,127.5406,8.063560,13.88701,0.522248,2.277803,0.249605,66.30219,1.018440,...,57.28217,0.000000,-999.000000,0.0,-999.0,121.120320,0.000000,1.675743,0.293980,0.172727
1,1,57.11760,119.5265,9.303685,15.14444,0.084197,3.726328,0.148465,69.82677,0.675294,...,185.99640,3.068812,3.711502,0.0,-999.0,54.125643,0.000000,3.821900,1.301525,2.492752
2,2,2.08080,119.5615,9.400000,15.11700,0.056098,4.004286,0.047523,70.00000,1.044123,...,90.00000,0.000000,-999.000000,0.0,-999.0,54.125643,0.000000,4.254696,0.302768,0.000000
3,3,493.16220,137.2501,5.894132,11.93295,1.371181,0.932440,0.999126,68.68895,0.316812,...,32.74243,6.214682,18.087680,0.0,-999.0,152.762449,0.027244,0.996138,42.000660,14.352010
4,4,1183.97300,114.7634,6.354364,11.96748,0.949577,0.215594,0.567381,67.10426,0.380921,...,42.80731,2.849865,11.538900,0.0,-999.0,161.210720,0.000000,1.353241,1.845117,9.432349


In [23]:
df_temp.to_feather('attr_temp99%_days_99sites.feather')

然后看看feather数据的读取：

In [24]:
train_data = pd.read_feather("attr_temp99%_days_99sites.feather")
train_data.head()

,Unnamed: 0,DRAIN_SQKM,PPTAVG_BASIN,T_AVG_BASIN,T_MAX_BASIN,T_MAXSTD_BASIN,T_MIN_BASIN,T_MINSTD_BASIN,RH_BASIN,STREAMS_KM_SQ_KM,...,CONTACT,CANALS_PCT,RAW_AVG_DIS_ALLCANALS,NPDES_MAJ_DENS,RAW_AVG_DIS_ALL_MAJ_NPDES,FRESHW_WITHDRAWAL,PCT_IRRIG_AG,ROADS_KM_SQ_KM,PADCAT1_PCT_BASIN,PADCAT2_PCT_BASIN
0,0,78.68936,127.5406,8.063560,13.88701,0.522248,2.277803,0.249605,66.30219,1.018440,...,57.28217,0.000000,-999.000000,0.0,-999.0,121.120320,0.000000,1.675743,0.293980,0.172727
1,1,57.11760,119.5265,9.303685,15.14444,0.084197,3.726328,0.148465,69.82677,0.675294,...,185.99640,3.068812,3.711502,0.0,-999.0,54.125643,0.000000,3.821900,1.301525,2.492752
2,2,2.08080,119.5615,9.400000,15.11700,0.056098,4.004286,0.047523,70.00000,1.044123,...,90.00000,0.000000,-999.000000,0.0,-999.0,54.125643,0.000000,4.254696,0.302768,0.000000
3,3,493.16220,137.2501,5.894132,11.93295,1.371181,0.932440,0.999126,68.68895,0.316812,...,32.74243,6.214682,18.087680,0.0,-999.0,152.762449,0.027244,0.996138,42.000660,14.352010
4,4,1183.97300,114.7634,6.354364,11.96748,0.949577,0.215594,0.567381,67.10426,0.380921,...,42.80731,2.849865,11.538900,0.0,-999.0,161.210720,0.000000,1.353241,1.845117,9.432349


另外还有一类数据文件格式也是常用的 -- json文件，在pandas中，读取json文件，可以使用 read_json() 

In [25]:
import pandas as pd

strtext='[{"ttery":"min","issue":"20130801-3391","code":"8,4,5,2,9","code1":"297734529","code2":null,"time":1013395466000},\
{"ttery":"min","issue":"20130801-3390","code":"7,8,2,1,2","code1":"298058212","code2":null,"time":1013395406000},\
{"ttery":"min","issue":"20130801-3389","code":"5,9,1,2,9","code1":"298329129","code2":null,"time":1013395346000},\
{"ttery":"min","issue":"20130801-3388","code":"3,8,7,3,3","code1":"298588733","code2":null,"time":1013395286000},\
{"ttery":"min","issue":"20130801-3387","code":"0,8,5,2,7","code1":"298818527","code2":null,"time":1013395226000}]'

text_data = pd.read_json(strtext)
text_data

,ttery,issue,code,code1,code2,time
0,min,20130801-3391,"8,4,5,2,9",297734529,NaN,1013395466000
1,min,20130801-3390,"7,8,2,1,2",298058212,NaN,1013395406000
2,min,20130801-3389,"5,9,1,2,9",298329129,NaN,1013395346000
3,min,20130801-3388,"3,8,7,3,3",298588733,NaN,1013395286000
4,min,20130801-3387,"0,8,5,2,7",298818527,NaN,1013395226000


In [26]:
json_file = "test.json"
# 当json文件中仅仅是够dict的一组key-valuie时，要使用type='series'来保证读取正确
text_json = pd.read_json(json_file, typ='series')
print(type(text_json.index[0]))
text_json

<class 'numpy.int64'>


1011000      H
1013500      F
1015800      C
1017000      C
1019000      H
1139000      R
1144000    HRS
dtype: object

但是这里发现index类型自动默认为int了，根据 [How to read index data as string with pandas.read_csv()?](https://stackoverflow.com/questions/35058435/how-to-read-index-data-as-string-with-pandas-read-csv) 的介绍（虽然是针对read_csv的，不过应该是一样的），这是pandas里面的一个小bug，没法直接一行代码指定index的数据类型， 如果采用先读取后转换的方式，那么最前面的0 是变不回来的：

In [27]:
text_json.index = text_json.index.map(str)
print(type(text_json.index[0]))
text_json

<class 'str'>


1011000      H
1013500      F
1015800      C
1017000      C
1019000      H
1139000      R
1144000    HRS
dtype: object

可以采用这种方式：先读到dict里面，然后再变成series，或者dataframe，这时候index就是str了

In [28]:
import json
with open(json_file, 'r') as fp:
    my_object = json.load(fp)
all_sites_purposes = pd.Series(my_object)
all_sites_purposes

01011000      H
01013500      F
01015800      C
01017000      C
01019000      H
01139000      R
01144000    HRS
dtype: object

读取数据或者后面对数据进行操作时，有时会想要进行数据类型转换。可以参考[Pandas数据类型转换的几个小技巧](https://zhuanlan.zhihu.com/p/35287822)

Pandas中进行数据类型转换有三种基本方法：

- 使用astype()函数进行强制类型转换，比如：dataset['col3'].astype('float')
- 自定义函数进行数据类型转换
- 使用Pandas提供的函数如to_numeric()、to_datetime()

当待转换列中含有不能转换的特殊值时(例如￥,ErrorValue,14n等)astype()函数将失效。
astype()函数有效的情形：

- 数据列中的每一个单位都能简单的解释为数字(2, 2.12等）
- 数据列中的每一个单位都是数值类型且向字符串object类型转换

Pandas的astype()函数和复杂的自定函数之间有一个中间段，那就是Pandas的一些辅助函数。这些辅助函数对于某些特定数据类型的转换非常有用(如to_numeric()、to_datetime())。

如果想要配置一些项目，比如忽略某些行，选择分隔符等，需要配置对应的参数，比如：

In [29]:
import os
import pandas as pd
fn = "TMaxMon.csv"
df = pd.read_csv(fn, engine='python', index_col=0, header=None,
                         skiprows=2, sep=',')
df.drop(list(df.columns)[0], axis=1, inplace=True)
df

,2,3,4,5,6,7,8,9,10,11,12,13
0,,,,,,,,,,,,
11120301,10.16,12.86,17.22,22.94,26.93,31.40,33.96,33.25,29.21,23.72,16.25,11.29
11120201,10.81,13.36,17.56,23.17,27.43,31.99,34.69,33.88,29.57,24.10,16.70,11.94
11120202,11.45,14.05,18.72,24.46,28.38,33.16,35.67,34.95,30.52,24.98,17.54,12.55
11130101,11.98,15.09,19.92,25.50,29.61,34.41,36.71,36.09,31.48,25.68,17.96,12.94
11120202,11.71,14.86,19.64,25.13,29.30,33.94,36.44,35.72,31.22,25.76,17.99,12.75
11120304,11.42,14.72,19.58,25.16,29.22,33.86,36.47,35.81,31.33,25.67,17.71,12.45
11120302,10.69,13.60,18.00,23.64,27.69,32.47,35.23,34.59,30.27,24.51,16.85,11.71
11120303,11.34,14.53,19.22,24.64,28.91,33.78,36.19,35.37,30.91,25.35,17.68,12.64


注意，pandas的Dataframe是没有原子性的，即行之间是可以完全相同的，所以这方面要注意，下面写入dict之后就会少一行

In [30]:
d = {}
for node_id, row in df.iterrows():
    print(node_id)
    d[node_id] = row.values.tolist()
print(d.keys())

11120301
11120201
11120202
11130101
11120202
11120304
11120302
11120303
dict_keys([11120301, 11120201, 11120202, 11130101, 11120304, 11120302, 11120303])


除了读取数据，就是写入文件了，写文件最常用的就是to_csv函数。

In [31]:
import pandas as pd
import numpy as np
df = pd.DataFrame({
    'col1': ['A', 'A', 'B', np.nan, 'D', 'C'],
    'col2': [2, 1, 9, 8, 7, 4],
    'col3': [0, 1, 9, 4, 2, 3],
})
file_name = "test.csv"
df.to_csv(file_name)

如果写入的时候不要带index，则设置index为False即可：

In [32]:
df.to_csv(file_name,index=False)

## 日期时间处理

和python基础，numpy一样，pandas也有自己的日期处理包，日期总是比较麻烦的。

In [33]:
import pandas as pd
# 日期／字符串转换
strtime=['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31']
time=pd.to_datetime(strtime)
[[dt.year, dt.month, dt.day, dt.hour] for dt in time]

[[2000, 1, 31, 0],
 [2000, 2, 29, 0],
 [2000, 3, 31, 0],
 [2000, 4, 30, 0],
 [2000, 5, 31, 0],
 [2000, 6, 30, 0],
 [2000, 7, 31, 0],
 [2000, 8, 31, 0],
 [2000, 9, 30, 0],
 [2000, 10, 31, 0]]

In [34]:
my_time = pd.DataFrame([[dt.year, dt.month, dt.day, dt.hour] for dt in time], columns=['Year', 'Mnth', 'Day', 'Hr'])
my_time.head()

,Year,Mnth,Day,Hr
0,2000,1,31,0
1,2000,2,29,0
2,2000,3,31,0
3,2000,4,30,0
4,2000,5,31,0


下面看看将数字转为日期。

例子的数据中前面三列是年月日，最后一列是数据：

In [40]:
# 数字，日期转换
import pandas as pd
data_temp = pd.DataFrame([[1952,1,1,3950],
[1952,1,2,3920],
[1952,1,3,3960],
[1952,1,4,3970]])
data_temp

,0,1,2,3
0,1952,1,1,3950
1,1952,1,2,3920
2,1952,1,3,3960
3,1952,1,4,3970


In [7]:
df_date = data_temp[[0, 1, 2]]
print(df_date)
df_date.columns = ['year', 'month', 'day']
date = pd.to_datetime(df_date).values.astype('datetime64[D]')
date

      0  1  2
0  1952  1  1
1  1952  1  2
2  1952  1  3
3  1952  1  4


array(['1952-01-01', '1952-01-02', '1952-01-03', '1952-01-04'],
      dtype='datetime64[D]')

## 排序

对DataFrame中的数据进行排序也是常用的操作之一。比如：

In [41]:
import pandas as pd
import numpy as np
df = pd.DataFrame({
    'col1': ['A', 'A', 'B', np.nan, 'D', 'C'],
    'col2': [2, 1, 9, 8, 7, 4],
    'col3': [0, 1, 9, 4, 2, 3],
})
df

,col1,col2,col3
0,A,2,0
1,A,1,1
2,B,9,9
3,NaN,8,4
4,D,7,2
5,C,4,3


Sort by col1

In [42]:
df.sort_values(by=['col1'])

,col1,col2,col3
0,A,2,0
1,A,1,1
2,B,9,9
5,C,4,3
4,D,7,2
3,NaN,8,4


Sort by multiple columns

In [43]:
df.sort_values(by=['col1', 'col2'])

,col1,col2,col3
1,A,1,1
0,A,2,0
2,B,9,9
5,C,4,3
4,D,7,2
3,NaN,8,4


排序这个地方需要注意下NaN的顺序是会比较奇怪的，可以看到上面NaN是被当作最大值了，这种数值条件下是一样的。

In [44]:
import pandas as pd
import numpy as np
df_nan = pd.DataFrame({'col1': [0, 2, 5, np.nan, 7, 10]})
df_nan

,col1
0,0.0
1,2.0
2,5.0
3,NaN
4,7.0
5,10.0


In [45]:
df_nan.sort_values(by=['col1'])

,col1
0,0.0
1,2.0
2,5.0
4,7.0
5,10.0
3,NaN


## 增删，拼接等操作

比如删除某些全是nan值的列。

In [46]:
import pandas as pd
import numpy as np
df0 = pd.DataFrame({"a":np.arange(3),"b":[1,np.nan,3],"c":[np.nan,np.nan,np.nan]})
df0

,a,b,c
0,0,1.0,NaN
1,1,NaN,NaN
2,2,3.0,NaN


直接删除的话，前面已经提到，直接drop对应列即可。

In [47]:
df1 = df0.drop(['c'], axis=1)
df1

,a,b
0,0,1.0
1,1,NaN
2,2,3.0


但如果想要批量判断，再删除，可以这样做：

In [48]:
# 判断哪些列全为nan
is_all_nan = df0.apply(lambda x: all(pd.isna(x)), axis=0)
is_all_nan

a    False
b    False
c     True
dtype: bool

In [49]:
# 删除其中为真的列
df2=df0.drop(is_all_nan[is_all_nan==True].index,axis=1)
df2

,a,b
0,0,1.0
1,1,NaN
2,2,3.0


可参考：[Merge, join, and concatenate](https://www.pypandas.cn/docs/user_guide/merging.html#concatenating-objects)

拼接主要使用的是concat函数，axis=0是默认选项纵向拼接，axis=1是横向

In [50]:
import pandas as pd
df0 = pd.DataFrame()
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                   'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                     index=[0, 1, 2, 3]) 

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                   'B': ['B4', 'B5', 'B6', 'B7'],
                  'C': ['C4', 'C5', 'C6', 'C7'],
                     'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7]) 

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                     'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11]) 

frames = [df0, df1, df2, df3]
result = pd.concat(frames)
print(result)

      A    B    C    D
0    A0   B0   C0   D0
1    A1   B1   C1   D1
2    A2   B2   C2   D2
3    A3   B3   C3   D3
4    A4   B4   C4   D4
5    A5   B5   C5   D5
6    A6   B6   C6   D6
7    A7   B7   C7   D7
8    A8   B8   C8   D8
9    A9   B9   C9   D9
10  A10  B10  C10  D10
11  A11  B11  C11  D11


In [51]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                        'D': ['D2', 'D3', 'D6', 'D7'],
                        'F': ['F2', 'F3', 'F6', 'F7']},
                       index=[2, 3, 6, 7])
result = pd.concat([df1, df4], axis=1)
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


再看 一个增加空列的例子，主要参考了：[pandas dataframe在指定的位置添加一列, 或者一次性添加几列，reindex，pd.concat的使用](https://blog.csdn.net/AlanGuoo/article/details/76522429)

In [52]:
df5 = pd.concat([df4, pd.DataFrame(columns=['flow', 'mode'])])
df5

,B,D,F,flow,mode
2,B2,D2,F2,NaN,NaN
3,B3,D3,F3,NaN,NaN
6,B6,D6,F6,NaN,NaN
7,B7,D7,F7,NaN,NaN


reindex可以来重新索引，如下所示，只用了df1的index，所以就剩下4行了

In [53]:
df5.reindex(df1.index)

,B,D,F,flow,mode
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,B2,D2,F2,NaN,NaN
3,B3,D3,F3,NaN,NaN


另外还有一个多列拼接成一列的小例子，几列字符串拼为一列字符串：

In [54]:
df = pd.DataFrame({"a": range(3), "b": range(3), "c": range(3)})
# 结构很简单: 第一列的名称.str.cat(第二列的名称)
df['a'] = df.iloc[:, 0].apply(str) + "-" + df['b'].apply(str) + "-" + df['b'].apply(str)
# 拼接之后，只留下特定的几列
df = df[['a', 'c']]
print(df)

       a  c
0  0-0-0  0
1  1-1-1  1
2  2-2-2  2


多个Series拼接

In [55]:
import pandas as pd
# 初始化的时候不起名，后面rename没有用
a = pd.Series([1, 2], name='aa')
rng1 = pd.date_range('2011-1-1', periods=2, freq='D')
print(type(rng1))
a.index = rng1
print("构建一个序列，并以时间做index：",a)
b = pd.Series([2, 3, 4])
rng2 = pd.date_range('2011-1-2', periods=3, freq='D')
b.index = rng2
df1 = pd.concat([a, b], axis=1)
print("拼接a和b：",df1)
c = pd.Series([5, 6])
rng3 = pd.date_range('2011-1-1', periods=2, freq='D')
c.index = rng3
df2 = pd.concat([df1, c], axis=1)
print("拼接df1和c：",df2)
# 尝试直接拼接多个：
df3 = pd.concat([df1, c, c], axis=1)
print("拼接多个目标：",df3)

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
构建一个序列，并以时间做index： 2011-01-01    1
2011-01-02    2
Freq: D, Name: aa, dtype: int64
拼接a和b：              aa    0
2011-01-01  1.0  NaN
2011-01-02  2.0  2.0
2011-01-03  NaN  3.0
2011-01-04  NaN  4.0
拼接df1和c：              aa    0    0
2011-01-01  1.0  NaN  5.0
2011-01-02  2.0  2.0  6.0
2011-01-03  NaN  3.0  NaN
2011-01-04  NaN  4.0  NaN
拼接多个目标：              aa    0    0    1
2011-01-01  1.0  NaN  5.0  5.0
2011-01-02  2.0  2.0  6.0  6.0
2011-01-03  NaN  3.0  NaN  NaN
2011-01-04  NaN  4.0  NaN  NaN


拼接还可以使用append：

In [56]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                       index=[0, 1, 2, 3])
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                        'B': ['B4', 'B5', 'B6', 'B7'],
                        'C': ['C4', 'C5', 'C6', 'C7'],
                        'D': ['D4', 'D5', 'D6', 'D7']},
                       index=[4, 5, 6, 7])
result = df1.append(df2)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


在数据处理中，经常会遇到空值地情况，这时候我们通常会进行插值，pandas中提供了非常方便地插值方式可供使用。

In [57]:
import pandas as pd
import numpy as np
s = pd.Series([0, 1, np.nan, 3])
s.interpolate()

0    0.0
1    1.0
2    2.0
3    3.0
dtype: float64

In [58]:
s

0    0.0
1    1.0
2    NaN
3    3.0
dtype: float64

注意interpolate函数并不是inplace操作。

如果需要平滑插值，可以使用样条插值，不过样条插值需要 scipy 包作为基础，后续会有scipy介绍，这里需要的话，可以先安装使用

```Shell
conda install -c conda-forge scipy
```

In [60]:
s = pd.Series([0, 2, np.nan, 8])
s.interpolate(method='polynomial', order=2)

0    0.000000
1    2.000000
2    4.666667
3    8.000000
dtype: float64

然后，对于DataFrame：

In [61]:
df = pd.DataFrame([(0.0, np.nan, -1.0, 1.0),
                   (np.nan, 2.0, np.nan, np.nan),
                   (2.0, 3.0, np.nan, 9.0),
                   (np.nan, 4.0, -4.0, 16.0)],
                  columns=list('abcd'))
df

,a,b,c,d
0,0.0,NaN,-1.0,1.0
1,NaN,2.0,NaN,NaN
2,2.0,3.0,NaN,9.0
3,NaN,4.0,-4.0,16.0


In [62]:
df.interpolate(method='linear', limit_direction='forward', axis=0)

,a,b,c,d
0,0.0,NaN,-1.0,1.0
1,1.0,2.0,-2.0,5.0
2,2.0,3.0,-3.0,9.0
3,2.0,4.0,-4.0,16.0


In [63]:
df.interpolate(method='linear', limit_direction='forward', axis=1)

,a,b,c,d
0,0.0,-0.5,-1.0,1.0
1,NaN,2.0,2.0,2.0
2,2.0,3.0,6.0,9.0
3,NaN,4.0,-4.0,16.0


## 修改操作

包括修改行列名，交换行列等操作。

行一般又称index；列名也称field。总体上可以分为在读数据时修改和读后修改两种。

具体的，行的修改方法有多种，参考：[重命名dataframe的index](https://blog.csdn.net/sinat_35930259/article/details/79872577)；列名的修改方法也有多种，参考[Pandas中修改DataFrame列名](http://www.voidcn.com/article/p-wycobfgd-bqs.html)

In [64]:
# 给出所有列名，这里类型是Index，可以直接转为list
import pandas as pd
from io import StringIO, BytesIO
data = ('col1,col2,col3\n'
         'a,b,01\n'
         'a,b,02\n'
         'c,d,03')
print("原数据：\n",data)
dataset = pd.read_csv(StringIO(data), index_col=0)
print("pandas读取默认设置：\n",dataset)
print(dataset.columns)
columns=dataset.columns.tolist()
print(columns)
# 转换后，每个元素直接就是字符串了
print(type(columns[0]))
# 修改列名的常用方式,inplace为false的话，还需要赋值到dataset，因此直接设置为true
dataset.rename(columns={'col1':'a', 'col2':'b', 'col3':'c'}, inplace = True)
print(dataset)
# 修改index行名

原数据：
 col1,col2,col3
a,b,01
a,b,02
c,d,03
pandas读取默认设置：
      col2  col3
col1           
a       b     1
a       b     2
c       d     3
Index(['col2', 'col3'], dtype='object')
['col2', 'col3']
<class 'str'>
      b  c
col1      
a     b  1
a     b  2
c     d  3


In [65]:
"""重新命名各列"""
new_col = ['new1', 'new2', 'new3', 'new4']
df2.columns = new_col
print(df2)

"""交换列的位置"""
order = ['new2', 'new1', 'new3', 'new4']
df2 = df2[order]
print(df2)

  new1 new2 new3 new4
4   A4   B4   C4   D4
5   A5   B5   C5   D5
6   A6   B6   C6   D6
7   A7   B7   C7   D7
  new2 new1 new3 new4
4   B4   A4   C4   D4
5   B5   A5   C5   D5
6   B6   A6   C6   D6
7   B7   A7   C7   D7


行列拆分，参考[DataFrame一列拆成多列以及一行拆成多行](https://blog.csdn.net/Asher117/article/details/84346073)，在数据分析时，经常需要把DataFrame的一列拆成多列或者根据某列把一行拆成多行。

In [66]:
import pandas as pd

df= pd.DataFrame({'Country': ['China', 'America', 'Japan'],
                   'City': ['Shanghai|Shenzhen', 'New York|State College', 'Tokyo|Osaka']},
                     index=[0, 1, 2])
df

,Country,City
0,China,Shanghai|Shenzhen
1,America,New York|State College
2,Japan,Tokyo|Osaka


In [67]:
df['Citys']=df['City'].map(lambda x: x.split('|'))
df

,Country,City,Citys
0,China,Shanghai|Shenzhen,"[Shanghai, Shenzhen]"
1,America,New York|State College,"[New York, State College]"
2,Japan,Tokyo|Osaka,"[Tokyo, Osaka]"


In [68]:
df['City1']=df['City'].map(lambda x: x.split('|')[0])
df['City2']=df['City'].map(lambda x: x.split('|')[1])
df

,Country,City,Citys,City1,City2
0,China,Shanghai|Shenzhen,"[Shanghai, Shenzhen]",Shanghai,Shenzhen
1,America,New York|State College,"[New York, State College]",New York,State College
2,Japan,Tokyo|Osaka,"[Tokyo, Osaka]",Tokyo,Osaka


除了上面的拆分，还有将各列拼接到一起，重新组织成符合数据库范式形式的表格，这就要用到十分常用的melt函数了，转换后的格式：其中一个或多个列是标识符变量，而所有其他列(被认为是测量变量)都不以行轴为轴，**只留下两个非标识符列**(变量和值)。

In [69]:
import pandas as pd
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                    'B': {0: 1, 1: 3, 2: 5},
                    'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [70]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [71]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [72]:
pd.melt(df, id_vars=['A'], value_vars=['B'], var_name='myVarname', value_name='myValname')

,A,myVarname,myValname
0,a,B,1
1,b,B,3
2,c,B,5


## 索引，切片等操作

首先是直接[]索引，以及loc和iloc的使用，这是pandas中最常用的索引方式。

切片包括横向切，纵向切，即选取某些行，选取某些列的操作。dataframe的切片操作很容易，记住灵活运用其index的功能即可，第一维是行，第二维是列，定位使用loc，数值定位用iloc。

先看看 Series 的索引操作

In [73]:
import numpy as np
import pandas as pd

se = pd.Series([1, 2, 3, 4], index=['A', 'B', 'C', 'D'])
se

A    1
B    2
C    3
D    4
dtype: int64

索引Series种符合条件的某些项可以直接采用如下方式

In [74]:
se[se>2]

C    3
D    4
dtype: int64

给出符号条件的项的索引

In [75]:
se[se==2].index.tolist()

['B']

再给一个例子，如果想要判断包含b的字符项对应的index，下面的方式是会报错的。

In [76]:
s1= pd.Series(['a', 'bv', "b", "d"], index=['A', 'B', 'C', 'D'])
s1["b" in s1].index.tolist()
# 或者
# s1[s1.find('b')].index.tolist()

KeyError: False

这时候需要使用apply函数遍历作用各项

In [77]:
include_b = s1.apply(lambda x:"b" in x)
s1[include_b==True].index.tolist()

['B', 'C']

In [78]:
data = pd.DataFrame(np.arange(16).reshape(4,4),index=list('abcd'),columns=list('wxyz'))
print(data)
print("选择表格中的'w'列，使用类字典属性,返回的是Series类型:", data['w'])

print("选择表格中的'w'列，使用点属性,返回的是Series类型:",data.w)

print("选择表格中的'w'列，返回的是DataFrame属性:",data[['w']])

print("选择表格中的'w'、'z'列:",data[['w','z']])  

print("返回第1行到第2行的所有行，前闭后开，包括前不包括后:",data[0:2])

print(" #返回第2行，从0计，返回的是单行，通过有前后值的索引形式:",data[1:2]) 
       #如果采用data[1]则报错

print("#返回第2行的第三种方法，返回的是DataFrame，跟data[1:2]同:",data.iloc[1:2]) 

print(" #利用index值进行切片，返回的是**前闭后闭**的DataFrame:",data['a':'b']) 

print("#返回data的前几行数据，默认为前五行，需要前十行则dta.head(10):",data.head())  
print("#返回data的后几行数据，默认为后五行，需要后十行则data.tail(10):",data.tail())  


print("#选取DataFrame最后一行，返回的是Series:",data.iloc[-1])   
print("#选取DataFrame最后一行，返回的是DataFrame:",data.iloc[-1:])  

print("#返回‘a’行'w'、'x'列，这种用于选取行索引列索引已知",data.loc['a',['w','x']])   

print("#选取第二行第二列，用于已知行、列位置的选取:",data.iat[1,1])   

    w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15
选择表格中的'w'列，使用类字典属性,返回的是Series类型: a     0
b     4
c     8
d    12
Name: w, dtype: int32
选择表格中的'w'列，使用点属性,返回的是Series类型: a     0
b     4
c     8
d    12
Name: w, dtype: int32
选择表格中的'w'列，返回的是DataFrame属性:     w
a   0
b   4
c   8
d  12
选择表格中的'w'、'z'列:     w   z
a   0   3
b   4   7
c   8  11
d  12  15
返回第1行到第2行的所有行，前闭后开，包括前不包括后:    w  x  y  z
a  0  1  2  3
b  4  5  6  7
 #返回第2行，从0计，返回的是单行，通过有前后值的索引形式:    w  x  y  z
b  4  5  6  7
#返回第2行的第三种方法，返回的是DataFrame，跟data[1:2]同:    w  x  y  z
b  4  5  6  7
 #利用index值进行切片，返回的是**前闭后闭**的DataFrame:    w  x  y  z
a  0  1  2  3
b  4  5  6  7
#返回data的前几行数据，默认为前五行，需要前十行则dta.head(10):     w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15
#返回data的后几行数据，默认为后五行，需要后十行则data.tail(10):     w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15
#选取DataFrame最后一行，返回的是Series: w    12
x    13
y    14
z    15
Name: d, dt

查看Dataframe各行信息：

In [79]:
print(data.index)
data.index.tolist()

Index(['a', 'b', 'c', 'd'], dtype='object')


['a', 'b', 'c', 'd']

查看Dataframe各列信息：

In [80]:
print(type(data.columns))
data.columns.tolist()

<class 'pandas.core.indexes.base.Index'>


['w', 'x', 'y', 'z']

如果某列用字符串索引，行用数字索引，那么需要这样做：

In [81]:
data = pd.DataFrame(np.arange(16).reshape(4,4),index=list('abcd'),columns=list('wxyz'))
print(data)
data['x'][0]

    w   x   y   z
a   0   1   2   3
b   4   5   6   7
c   8   9  10  11
d  12  13  14  15


1

In [82]:
"""取dataframe指定多行列 slice操作"""
# 取多行
print(df2.iloc[0:2])
# 取多列
print(df2.iloc[:, 0:2])
# 取多行多列
print(df2.iloc[0:2, 0:2])

  new2 new1 new3 new4
4   B4   A4   C4   D4
5   B5   A5   C5   D5
  new2 new1
4   B4   A4
5   B5   A5
6   B6   A6
7   B7   A7
  new2 new1
4   B4   A4
5   B5   A5


series的slice

In [83]:
"""给Series作slice操作"""
arr = [1, 2, 3, 4]  # 创建数组
series_1 = pd.Series(arr)
series_1.index = ['a', 'b', 'c', 'd']
print("------------------Series查询操作----------------------")
print(series_1['a'])
print(series_1[['a', 'b']])
print(series_1[series_1 > 2])
print(series_1[:2])
print(series_1['a':'c'])

------------------Series查询操作----------------------
1
a    1
b    2
dtype: int64
c    3
d    4
dtype: int64
a    1
b    2
dtype: int64
a    1
b    2
c    3
dtype: int64


在dataframe中还常用到条件筛选。参考[30分钟带你入门数据分析工具 Pandas（上篇），果断收藏](https://zhuanlan.zhihu.com/p/44174554)，用中括号 [] 的方式，除了直接指定选中某些列外，还能接收一个条件语句，然后筛选出符合条件的行/列。比如：

In [84]:
import pandas as pd
import numpy as np
df=pd.DataFrame(np.random.randn(5,4),['A','B','C','D','E'],['W','X','Y','Z'])
print(df)
# 筛选出 'W'>0 的行
print(df[df['W']>0])
print('\n')
# 只看 'X' 列中 'W'>0 的数据
print(df[df['W']>0]['X'])
print('\n')
print(df[df['W']>0][['X','Y']])

          W         X         Y         Z
A  1.444328 -1.473487 -1.659399  0.298594
B  0.530866  0.566232  0.987480 -0.931774
C  0.143968  0.958281  0.786524 -0.032190
D  0.620018 -0.192263 -0.175855 -1.118276
E -0.180265  0.438621  2.173577 -2.866212
          W         X         Y         Z
A  1.444328 -1.473487 -1.659399  0.298594
B  0.530866  0.566232  0.987480 -0.931774
C  0.143968  0.958281  0.786524 -0.032190
D  0.620018 -0.192263 -0.175855 -1.118276


A   -1.473487
B    0.566232
C    0.958281
D   -0.192263
Name: X, dtype: float64


          X         Y
A -1.473487 -1.659399
B  0.566232  0.987480
C  0.958281  0.786524
D -0.192263 -0.175855


如果想要给符合条件的某些行的某列赋值，可以使用如下形式，顺便再理解下loc，loc的参数一个表示行索引，一个表示列索引。这是一个inplace的操作：

In [85]:
df.loc[df['W']>0, 'W'] = 1
df

,W,X,Y,Z
A,1.000000,-1.473487,-1.659399,0.298594
B,1.000000,0.566232,0.987480,-0.931774
C,1.000000,0.958281,0.786524,-0.032190
D,1.000000,-0.192263,-0.175855,-1.118276
E,-0.180265,0.438621,2.173577,-2.866212


还可以用逻辑运算符 &（与）和 |（或）来链接多个条件语句，以便一次应用多个筛选条件到当前的 DataFrame 上，比如筛选出同时满足 'W'>0 和'X'>1 的行：

In [86]:
df[(df['W']>0) & (df['X']<1)]

,W,X,Y,Z
A,1.0,-1.473487,-1.659399,0.298594
B,1.0,0.566232,0.987480,-0.931774
C,1.0,0.958281,0.786524,-0.032190
D,1.0,-0.192263,-0.175855,-1.118276


如果想要获取符合条件的行的索引也很简单，如下所示：

In [87]:
df[(df['W']>0) & (df['X']<1)].index.tolist()

['A', 'B', 'C', 'D']

再比如，还可以让列之间进行比较并作为索引条件

In [88]:
df

,W,X,Y,Z
A,1.000000,-1.473487,-1.659399,0.298594
B,1.000000,0.566232,0.987480,-0.931774
C,1.000000,0.958281,0.786524,-0.032190
D,1.000000,-0.192263,-0.175855,-1.118276
E,-0.180265,0.438621,2.173577,-2.866212


In [89]:
df[df['W']<df['X']]

,W,X,Y,Z
E,-0.180265,0.438621,2.173577,-2.866212


有一种常见的索引形式，即选出Dataframe中某一列在一个list中的那些项：

In [90]:
import pandas as pd

df = pd.DataFrame({'A': [5,6,3,4], 'B': [1,2,3,5]})
df

,A,B
0,5,1
1,6,2
2,3,3
3,4,5


In [91]:
df[df['A'].isin([3, 6])]

,A,B
1,6,2
2,3,3


如果想要不在某个list中的项：

In [92]:
df[~df['A'].isin([3, 6])]

,A,B
0,5,1
3,4,5


使用pandas的loc时要注意，当有missing labels，在pandas 1.0版本之后是会报错的，更推荐的做法是使用reindex()

In [93]:
import pandas as pd
s = pd.Series([1, 2, 3])
s.loc[[1, 2]]

1    2
2    3
dtype: int64

In [94]:
s.loc[[1, 2, 3]] # missing value使用loc会报错

KeyError: '[3] not in index'

现在看看使用reindex，更多内容可以参考：https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike

In [95]:
s.reindex([1, 2, 3])

1    2.0
2    3.0
3    NaN
dtype: float64

如果仅仅想要去除valid keys对应的值，可以使用：

In [96]:
labels = [1, 2, 3]
s.index.intersection(labels)
# s.loc[s.index.intersection(labels)]

Int64Index([1, 2], dtype='int64')

另外一个需要注意的是NaN的索引，pandas里面是容易犯错的。参考：https://blog.csdn.net/S_o_l_o_n/article/details/100661937

pandas基于numpy，所以其中的空值nan和numpy.nan是等价的。numpy中的nan并不是空对象，其实际上是numpy.float64对象，所以我们不能误认为其是空对象，从而用bool(np.nan)去判断是否为空值，这是不对的。

对于pandas中的空值，我们该如何判断，并且有哪些我们容易掉进去的陷阱，即不能用怎么样的方式去判断呢？

可以判断pandas中单个空值对象的方式：

1. 利用pd.isnull(),pd.isna();
2. 利用np.isnan();
3. 利用is表达式；
4. 利用in表达式。

不可以用来判断pandas单个空值对象的方式：

1. **不可**直接用==表达式判断；
2. **不可**直接用bool表达式判断；
3. **不可**直接用if语句判断。

对于同时多个空值对象的判断和处理：

1. 可以用Series对象和DataFrame对象的any()或all()方法；
2. 可以用numpy的any()或all()方法；
3. 不可以直接用python的内置函数any()和all()方法；
4. 可以用Series或DataFrame对象的dropna()方法剔除空值；
5. 可以用Series或DataFrame对象的fillna()方法填充空值。

In [97]:
import pandas as pd
import numpy as np
 
na=np.nan
 
# 可以用来判断空值的方式
print(pd.isnull(na))  # True
print(pd.isna(na))  # True
print(np.isnan(na))  # True
print(na is np.nan)  # True
print(na in [np.nan])  # True
 
 
# 不可以直接用来判断的方式，即以下结果和我们预期不一样
print(na == np.nan)  # False
print(bool(na))  # True
if na:
    print('na is not null')  # Output: na is not null
 
 
# 不可以直接用python内置函数any和all
print(any([na])) # True
print(all([na]))  #True

True
True
True
True
True
False
True
na is not null
True
True


In [98]:
df_nan = pd.DataFrame({"a":[1,2,3,np.nan,4,5,6,7,8,9]})
df_nan.dropna() # 不是内置运算，即不改变原来的dataframe

,a
0,1.0
1,2.0
2,3.0
4,4.0
5,5.0
6,6.0
7,7.0
8,8.0
9,9.0


In [99]:
df_nan

,a
0,1.0
1,2.0
2,3.0
3,NaN
4,4.0
5,5.0
6,6.0
7,7.0
8,8.0
9,9.0


查找 nan 所在位置：

In [100]:
df_nan[df_nan.values!=df_nan.values] 

,a
3,NaN


In [101]:
# 给出索引
df_nan[df_nan.values!=df_nan.values].index.tolist()

[3]

## 分组

对dataframe进行分组的例子。

任何分组(groupby)操作都涉及原始对象的以下操作之一。它们是 - 

- 分割对象
- 应用一个函数
- 结合的结果

在许多情况下，我们将数据分成多个集合，并在每个子集上应用一些函数。在应用函数中，可以执行以下操作 -

- 聚合 - 计算汇总统计
- 转换 - 执行一些特定于组的操作
- 过滤 - 在某些情况下丢弃数据

In [102]:
salaries = pd.DataFrame({
    'name': ['BOSS', 'Lilei', 'Lilei', 'Han', 'BOSS', 'BOSS', 'Han', 'BOSS'],
    'Year': [2016, 2016, 2016, 2016, 2017, 2017, 2017, 2017],
    'Salary': [999999, 20000, 25000, 3000, 9999999, 999999, 3500, 999999],
    'Bonus': [100000, 20000, 20000, 5000, 200000, 300000, 3000, 400000]
})
print(salaries.columns)
print(salaries.info())
print(salaries.describe())
salaries = salaries[['name', 'Year', 'Salary', 'Bonus']]
# 定顺序
print(salaries)
# 对dataframe按name进行分组
group_by_name = salaries.groupby('name')
# 获取分组后的某一组
se_temp = group_by_name.get_group('Lilei')
print(se_temp)
print(se_temp.describe())
# 循环各组，并将名字在给定的序列中的group添加到数组中
config = pd.Series({'names': ['Lilei', 'BOSS']})
dfs = []
for name, group in group_by_name:
    print(name)
    if name in config['names']:
        dfs.append(group)

for i in range(len(dfs)):
    # 如果没有把name和group分开，那么dfs[i]的类型会是tuple，key为name，value是group，可参考接下来的输出
    print(type(dfs[i]))
    print(dfs[i])

# 如果没有把name和group分开，那么dfs[i]的类型会是tuple，key为name，value是group
dfs_s = []
for group in group_by_name:
    dfs_s.append(group)
for i in range(len(dfs_s)):
    print(type(dfs_s[i]))
    print(dfs_s[i])

Index(['name', 'Year', 'Salary', 'Bonus'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    8 non-null      object
 1   Year    8 non-null      int64 
 2   Salary  8 non-null      int64 
 3   Bonus   8 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 384.0+ bytes
None
              Year        Salary          Bonus
count     8.000000  8.000000e+00       8.000000
mean   2016.500000  1.631437e+06  131000.000000
std       0.534522  3.416521e+06  152851.935826
min    2016.000000  3.000000e+03    3000.000000
25%    2016.000000  1.587500e+04   16250.000000
50%    2016.500000  5.124995e+05   60000.000000
75%    2017.000000  9.999990e+05  225000.000000
max    2017.000000  9.999999e+06  400000.000000
    name  Year   Salary   Bonus
0   BOSS  2016   999999  100000
1  Lilei  2016    20000   20000
2  Lilei  2016    25000   20000
3    

如果想要将类型数据转换为数字，则可以使用factorize函数：

In [103]:
# 返回列数：
print(dataset.shape[1])
# 返回行数：
print(dataset.shape[0])
# factorize(values[, sort, order, …])	Encode the object as an enumerated type or categorical variable.
labels, uniques = pd.factorize(['b', 'b', 'a', 'c', 'b'])
print(labels)
uniques

2
3
[0 0 1 2 0]


array(['b', 'a', 'c'], dtype=object)

聚合函数为每个组返回单个聚合值。当创建了分组(group by)对象，就可以对分组数据执行多个聚合操作。

In [104]:
import pandas as pd
import numpy as np

ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [105]:
grouped = df.groupby('Year')
# .round(2) 是保留两位小数
grouped['Points'].agg(np.mean).round(2)

Year
2014    795.25
2015    769.50
2016    725.00
2017    739.00
Name: Points, dtype: float64

## 批量运算

Pandas中，为了加速运算，通常会尽量采用向量化的方式进行运算。

对dataframe数据执行批量运算，使用pandas的apply函数。

In [106]:
import pandas as pd
import numpy as np
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
print(df)
df.apply(np.sqrt, axis=1)

   A  B
0  4  9
1  4  9
2  4  9


,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


apply函数默认是作用到dataframe每列，axis=0

In [107]:
df.apply(np.sum)

A    12
B    27
dtype: int64

In [108]:
df.apply(lambda x:sum(x))

A    12
B    27
dtype: int64

In [109]:
df.apply(lambda x: x**2)

,A,B
0,16,81
1,16,81
2,16,81


如果想要作用于某些指定的列，比如对第二列之后的运算，可以如下操作：

In [110]:
df = pd.DataFrame([[1,2,3,4,5,6]] * 3, columns=['ID', 'A', 'B', 'C', 'D', 'E'])
df

,ID,A,B,C,D,E
0,1,2,3,4,5,6
1,1,2,3,4,5,6
2,1,2,3,4,5,6


In [111]:
df.iloc[:,1:].apply(lambda x: x**2)

,A,B,C,D,E
0,4,9,16,25,36
1,4,9,16,25,36
2,4,9,16,25,36


也可以直接指定某些列：

In [112]:
df[["A","B","C"]].apply(lambda x:sum(x))

A     6
B     9
C    12
dtype: int64

如果想要对一行去做运算，需要指定 axis=1

In [113]:
df[["A","B","C"]].apply(lambda x:sum(x),axis=1)

0    9
1    9
2    9
dtype: int64

如果想要对一列的每个值操作一个函数，那么可以：

In [114]:
df["A"].apply(np.sqrt)

0    1.414214
1    1.414214
2    1.414214
Name: A, dtype: float64

但是可以看到，这并不是一个原地运算的函数，df是没变的：

In [115]:
df

,ID,A,B,C,D,E
0,1,2,3,4,5,6
1,1,2,3,4,5,6
2,1,2,3,4,5,6


如果想要执行inplace运算，则需要重新赋值一下：

In [116]:
df["A"]=df["A"].apply(np.sqrt)
df

,ID,A,B,C,D,E
0,1,1.414214,3,4,5,6
1,1,1.414214,3,4,5,6
2,1,1.414214,3,4,5,6


如果有nan值，可以使用忽略nan值的运算

In [117]:
df = pd.DataFrame({'A':[1,2,3,np.nan], 'B':[np.nan,2,np.nan,6]})
df.apply(lambda x:np.nansum(x), axis=1).values

array([1., 4., 3., 6.])

如果要指定的函数返回值和原dataframe不一致，为了继续使用向量化以使代码速度较快，可以利用numpy的相关函数

In [118]:
import numpy as np
from scipy import interpolate
num_of_years = 20
df = pd.DataFrame([[1,2,3,4,5,6]] * 3, columns=['A', 'B', 'C', 'D', 'E', 'F'])
data_np=df.values
data_np

array([[1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6]], dtype=int64)

In [119]:
def interpolate_myself(y):
    x = np.linspace(0, num_of_years - 1, num=6)
    xs = np.linspace(0, num_of_years - 1, num=num_of_years)
    ys = interpolate.UnivariateSpline(x, y, s=0)(xs)
    return ys
np_new=np.apply_along_axis(interpolate_myself, 1, data_np)
np_new

array([[1.        , 1.26315789, 1.52631579, 1.78947368, 2.05263158,
        2.31578947, 2.57894737, 2.84210526, 3.10526316, 3.36842105,
        3.63157895, 3.89473684, 4.15789474, 4.42105263, 4.68421053,
        4.94736842, 5.21052632, 5.47368421, 5.73684211, 6.        ],
       [1.        , 1.26315789, 1.52631579, 1.78947368, 2.05263158,
        2.31578947, 2.57894737, 2.84210526, 3.10526316, 3.36842105,
        3.63157895, 3.89473684, 4.15789474, 4.42105263, 4.68421053,
        4.94736842, 5.21052632, 5.47368421, 5.73684211, 6.        ],
       [1.        , 1.26315789, 1.52631579, 1.78947368, 2.05263158,
        2.31578947, 2.57894737, 2.84210526, 3.10526316, 3.36842105,
        3.63157895, 3.89473684, 4.15789474, 4.42105263, 4.68421053,
        4.94736842, 5.21052632, 5.47368421, 5.73684211, 6.        ]])

最后还需要还原为dataframe的话，执行DataFrame的初始化操作即可：

In [120]:
df2 = pd.DataFrame(np_new)
time_range = np.arange(1985,2005)
df2.columns = time_range
df2

,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004
0,1.0,1.263158,1.526316,1.789474,2.052632,2.315789,2.578947,2.842105,3.105263,3.368421,3.631579,3.894737,4.157895,4.421053,4.684211,4.947368,5.210526,5.473684,5.736842,6.0
1,1.0,1.263158,1.526316,1.789474,2.052632,2.315789,2.578947,2.842105,3.105263,3.368421,3.631579,3.894737,4.157895,4.421053,4.684211,4.947368,5.210526,5.473684,5.736842,6.0
2,1.0,1.263158,1.526316,1.789474,2.052632,2.315789,2.578947,2.842105,3.105263,3.368421,3.631579,3.894737,4.157895,4.421053,4.684211,4.947368,5.210526,5.473684,5.736842,6.0
